In [1]:
from typing import Iterable, List, Tuple

def solve_cnf(clauses: Iterable[Iterable[int]], num_vars: int) -> Tuple[str, List[int] | None]:
    """
    Implement your SAT solver here.
    Must return:
      ("SAT", model)  where model is a list of ints (DIMACS-style), or
      ("UNSAT", None)
    """
    raise NotImplementedError

In [ ]:
clauses = [[9, 3],[9],[4,2],[-9,4],[5,7],[1,-1],[1,3],[-2,-7]] # Truth list is 9,4,3,5

In [ ]:
def DLIS(clauses: Iterable[Iterable[int]], num_vars: int) -> Tuple[str, List[int] | None]:
    """
    Implement your DLIS SAT solver here.
    Must return:
      ("SAT", model)  where model is a list of ints (DIMACS-style), or
      ("UNSAT", None)

      CP(V) is the number of occurences of variable V in the clauses.
      CN(V) is the number of occurences of the negation of variable V in the clauses.
    """
    f = open(clauses, 'r')
    sudoku = f.read()
    splitlines = sudoku.splitlines()
    N = len(splitlines)
    clauses = list()
    

DLIS("output.cnf", 729)
    

1 2 3 4 5 6 7 8 9 0


In [2]:
from typing import List

def parse_dimacs(source: str) -> List[List[int]]:
    """
    Parse DIMACS CNF from a string or a file path and return a list of clauses.
    Each clause is a list of ints, e.g. [[1, -2], [-1, 3]].

    If `source` is a path to a file, the file will be read. Otherwise `source`
    is treated as the DIMACS content.
    """
    try:
        # try treating source as a file path
        with open(source, 'r') as f:
            text = f.read()
    except (OSError, TypeError):
        # treat source as DIMACS content string
        text = source

    clauses: List[List[int]] = []
    current_clause: List[int] = []

    for line in text.splitlines():
        line = line.strip()
        if not line or line.startswith('c') or line.startswith('p'):
            continue
        for tok in line.split():
            try:
                lit = int(tok)
            except ValueError:
                # skip any non-integer tokens (robustness)
                continue
            if lit == 0:
                if current_clause:
                    clauses.append(current_clause)
                    current_clause = []
            else:
                current_clause.append(lit)

    # if file omitted final 0, append any remaining clause
    if current_clause:
        clauses.append(current_clause)

    return clauses

parse_dimacs("output.cnf")

[[1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18],
 [19, 20, 21, 22, 23, 24, 25, 26, 27],
 [28, 29, 30, 31, 32, 33, 34, 35, 36],
 [37, 38, 39, 40, 41, 42, 43, 44, 45],
 [46, 47, 48, 49, 50, 51, 52, 53, 54],
 [55, 56, 57, 58, 59, 60, 61, 62, 63],
 [64, 65, 66, 67, 68, 69, 70, 71, 72],
 [73, 74, 75, 76, 77, 78, 79, 80, 81],
 [82, 83, 84, 85, 86, 87, 88, 89, 90],
 [91, 92, 93, 94, 95, 96, 97, 98, 99],
 [100, 101, 102, 103, 104, 105, 106, 107, 108],
 [109, 110, 111, 112, 113, 114, 115, 116, 117],
 [118, 119, 120, 121, 122, 123, 124, 125, 126],
 [127, 128, 129, 130, 131, 132, 133, 134, 135],
 [136, 137, 138, 139, 140, 141, 142, 143, 144],
 [145, 146, 147, 148, 149, 150, 151, 152, 153],
 [154, 155, 156, 157, 158, 159, 160, 161, 162],
 [163, 164, 165, 166, 167, 168, 169, 170, 171],
 [172, 173, 174, 175, 176, 177, 178, 179, 180],
 [181, 182, 183, 184, 185, 186, 187, 188, 189],
 [190, 191, 192, 193, 194, 195, 196, 197, 198],
 [199, 200, 201, 202, 203, 204, 205, 206, 207],
 [20

In [ ]:
def dlis_pick_literal(clauses, assignment):
    score = {}  # dict: literal -> int

    for clause in clauses:
        if clause_is_satisfied(clause, assignment):
            continue  # ignore satisfied clauses

        # Also skip clauses that are already conflicting given the assignment,
        # but usually that’s handled earlier in propagation.

        for lit in clause:
            if literal_is_false(lit, assignment):
                continue  # assigning this literal won't satisfy the clause now
            if literal_is_true(lit, assignment):
                # then the clause is satisfied, wouldn't have come here normally,
                # but you can handle it as needed.
                continue

            # At this point, lit is unassigned in this clause
            score[lit] = score.get(lit, 0) + 1

    if not score:
        return None  # no literal to branch on

    # pick literal with maximum score
    best_lit = max(score, key=score.get)
    return best_lit  # this literal will be set to True


In [13]:
cnf = [
    [1, -2],   # (x1 ∨ ¬x2)
    [-1, 3],   # (¬x1 ∨ x3)
    ]


In [3]:
import itertools

def eval_cnf(cnf, assignment):
    """
    cnf: lijst van clausules, bijv. [[1, -2], [-1, 3]]
    assignment: dict met var -> bool, bijv. {1: True, 2: False, 3: True}
    """
    for clause in cnf:
        # Een clausule is waar als minstens één literal waar is
        clause_true = False
        for lit in clause:

            var = abs(lit)
            val = assignment[var]  # True of False
            if lit > 0 and val:       # x is waar
                clause_true = True
                break
            if lit < 0 and not val:   # ¬x is waar
                clause_true = True
                break

        if not clause_true:
            # Eén clausule is onwaar -> hele CNF is onwaar
            return False

    # Alle clausules zijn waar
    return True


def sat_solve(cnf):
    """
    Probeer alle mogelijke toekenningen (brute force).
    Geeft een assignment dict terug als de CNF satisfiable is, anders None.
    """
    # Verzamel alle variabelen die in de CNF voorkomen
    vars_set = {abs(lit) for clause in cnf for lit in clause}
    vars_list = sorted(vars_set)
    # print("Variabelen:", vars_list)

    # Genereer alle combinaties van True/False voor deze variabelen
    for values in itertools.product([True, False], repeat=len(vars_list)):
        assignment = dict(zip(vars_list, values))
        # print("Proberen:", assignment)
        if eval_cnf(cnf, assignment):
            return assignment  # gevonden!

    # Niets gevonden -> unsatisfiable
    return None



def solving(cnf):
    solution = sat_solve(cnf)
    if solution is None:
        print("Niet satisfiable")
    else:
        print("Satisfiable, voorbeeld-toekenning:")
        for var, val in sorted(solution.items()):
            print(f"x{var} = {val}")

solving(parse_dimacs("output.cnf"))

KeyboardInterrupt: 

In [ ]:
from typing import List, Tuple, Dict, Optional

def dlis_pick_literal(clauses: List[List[int]], assignment) -> Optional[int]:
    score: Dict[int, int] = {}

    for clause in clauses:
            # skip satisfied clauses
            if any(literal_value(lit, assignment) is True for lit in clause):
                continue

            # for unsatisfied clauses, count unassigned literals
            for lit in clause:
                val = literal_value(lit, assignment)
                if val is None:  # unassigned
                    score[lit] = score.get(lit, 0) + 1

    if not score:
        return None

    best_lit = max(score.items(), key=lambda kv: kv[1])[0]
    return best_lit




In [ ]:
def makeScores(clauses, assignments) -> Tuple[bool, Assignment]:

In [ ]:
#!/usr/bin/env python3
"""
SAT Assignment Part 1 - Non-consecutive Sudoku Encoder (Puzzle -> CNF)

Do NOT modify this file - instead, implement your function in encoder.py

Usage:
  python main.py --in <puzzle.txt> --out <instance.cnf>
"""

import argparse
import sys
from encoder import to_cnf  #implement


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--in", dest="inp", required=True, help="Path to puzzle .txt")
    p.add_argument("--out", dest="out", default=None, help="Path to write DIMACS CNF (stdout if omitted)")
    return p.parse_args()


def main():
    args = parse_args()
    clauses, num_vars = to_cnf(args.inp)
    if args.out:
        write_dimacs(args.out, num_vars, clauses)
    else:
        write_dimacs(sys.stdout, num_vars, clauses)


if __name__ == "__main__":
    main()


In [ ]:
from typing import List, Tuple, Dict, Optional

Assignment = Dict[int, bool]

def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: list[list[int]]) -> list[list[int]]:
    # CNF moet gesolved worden:
    truth_list = list()
    # Step 1: simplify
    terminate = 0
    while not terminate: # Terminate when clauses is simplified
        taut_tot = 0
        pure_tot = 0
        unit_tot = 0
        # maybe a clauses copy?
        for clause in clauses[:]:

            # We are deleting from the clauses list so skipp a clause that has already been delete
            if not clause in clauses: continue

            # Tautology check
            taut = delete_tautologies(clause, clauses)
            taut_tot += taut
            if taut == 1:
                clauses.remove(clause)
                continue
            # Only continue in this clause if clause is not a tautology

            # Unit clause check
            clauses, truth, unit = unit_clause(clause, clauses)
            unit_tot += unit
            if not truth == 0: # Unit literal was found
                if not truth in truth_list:
                    truth_list.append(truth)
                    continue # Only pure literal check needed if no unit literal was found -> no double work

            # Pure Literal check
            clauses, truth = pure_literal(clause, clauses)
            pure_tot += len(truth)
            if not truth == list():
                for truth_item in truth_list:
                    if not truth_item in truth:
                        truth_list.append(truth_item)

        # Check if all are taken out of clauses
        # print(f"taut_tot: {taut_tot}, pure_tot: {pure_tot}, unit_tot: {unit_tot}")
        # print(f"truth_list: {truth_list} ")
        # print(f"and clauses: {clauses}")

        if taut_tot == 0 and pure_tot == 0 and unit_tot == 0: # ... because then nothing more to simplify
            terminate = 1


    return clauses, truth_list





def lit_var(lit: int) -> int:
    return abs(lit)


def literal_value(lit: int, assignment: Assignment) -> Optional[bool]:
    v = abs(lit)
    if v not in assignment:
        return None
    val = assignment[v]
    return val if lit > 0 else (not val)



def dlis_pick_literal(clauses: List[List[int]], assignment: Assignment) -> Optional[int]:
    score: Dict[int, int] = {}

    for clause in clauses:
        # skip satisfied clauses
        if any(literal_value(lit, assignment) is True for lit in clause):
            continue

        # for unsatisfied clauses, count unassigned literals
        for lit in clause:
            val = literal_value(lit, assignment)
            if val is None:  # unassigned
                score[lit] = score.get(lit, 0) + 1

    if not score:
        return None

    best_lit = max(score.items(), key=lambda kv: kv[1])[0]
    return best_lit


def first_unassigned_literal(clauses: List[List[int]], assignment: Assignment) -> Optional[int]:
    for clause in clauses:
        for lit in clause:
            v = lit_var(lit)
            if v not in assignment:
                return lit
    return None


def pick_literal(
    clauses: List[List[int]],
    assignment: Assignment,
    heuristic: str = "dlis"
) -> Optional[int]:
    if heuristic == "dlis":
        return dlis_pick_literal(clauses, assignment)
    elif heuristic == "first":
        return first_unassigned_literal(clauses, assignment)
    else:
        raise ValueError(f"Unknown heuristic: {heuristic}")


def dpll(
    clauses: List[List[int]],
    assignment: Optional[Assignment] = None,
    heuristic: str = "dlis"
) -> Tuple[bool, Optional[Assignment]]:
    """
    clauses: CNF as list of clauses (each clause is a list of ints)
    assignment: current partial assignment {var: bool}
    heuristic: "dlis" or "first"
    Returns (is_sat, model)
    """
    if assignment is None:
        assignment = {}

    # 1) Simplify with your procedure (tautologies, unit, pure literals, ...)
    clauses, implied_truths = simplify(clauses)

    # 2) Add implied truths to the assignment
    for lit in implied_truths:
        v = abs(lit)
        val = lit > 0
        # Optionally: check for inconsistency with existing assignment
        if v in assignment and assignment[v] != val:
            return False, None
        assignment[v] = val

    # 3) Check for empty clause -> UNSAT
    if any(len(c) == 0 for c in clauses):
        return False, None

    # 4) If no clauses left, SAT
    if not clauses:
        return True, assignment

    # 5) Choose splitting literal
    lit = pick_literal(clauses, assignment, heuristic)
    if lit is None:
        # all variables assigned, but we didn't satisfy all clauses -> UNSAT
        return False, None

    # 6) Branch: first try lit = True (add [lit] as a unit clause)
    clauses_true = clauses + [[lit]]
    sat, model = dpll(clauses_true, assignment.copy(), heuristic)
    if sat:
        return True, model

    # 7) If that fails, try lit = False (add [-lit])
    clauses_false = clauses + [[-lit]]
    return dpll(clauses_false, assignment.copy(), heuristic)



if __name__ == "__main__":
    cnf = [
        [1, -2],
        [-1, 3],
    ]

    is_sat, model = dpll(cnf, heuristic="dlis")  # or "first"
    if is_sat:
        print("SAT")
        print(model)
    else:
        print("UNSAT")
